<a href="https://colab.research.google.com/github/Pushkaran-P/Age-Race-Classification/blob/main/Age_Padding_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unrar x "/content/drive/MyDrive/PadTrainingData.rar"
!unrar x "/content/drive/MyDrive/PadTestingData.rar"

Streaming output truncated to the last 5000 lines.
Extracting  TrainingData/3/24_1_4_20170102233433610.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103212802644.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103212911364.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103212924934.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223129750.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223158047.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223727135.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223907208.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223911478.jpg                   71%  OK 
Extracting  TrainingData/3/24_1_4_20170103223935080.jpg                   71%  OK 
Extracting  TrainingData/3/24

In [3]:
import numpy as np
import os

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
train_output_dir = '/content/TrainingData'
#test_output_dir = 'D:/Studies/Age Classification Project/TestingData'

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_output_dir,
    batch_size=32,
    class_mode='sparse',  # If you have more than two classes
    target_size = (642,641),
    shuffle=True,
    subset='training'  # Specify this is training data
)

num_classes = train_generator.num_classes

test_datagen = ImageDataGenerator(rescale=1./255)
test_output_dir = '/content/TestingData'
#test_output_dir = 'D:/Studies/Age Classification Project/TestingData'

# Load the training data
test_generator = test_datagen.flow_from_directory(
    test_output_dir,
    batch_size=32,
    class_mode='sparse',  # If you have more than two classes
    target_size = (642,641),
    #subset='training'
)

Found 6676 images belonging to 5 classes.
Found 2989 images belonging to 5 classes.


##GPU and MacroF1 class

In [5]:
dim1 = train_generator.image_shape[0]
dim2 = train_generator.image_shape[1]
dim3 = train_generator.image_shape[2]

In [6]:
import tensorflow as tf

# Detect and initialize GPU
try:
    strategy = tf.distribute.OneDeviceStrategy('GPU')
except tf.errors.NotFoundError:
    print('No GPU devices found. Please make sure GPU is enabled in the runtime settings.')
    strategy = tf.distribute.get_strategy()

In [7]:
def macro_f1(y_true, y_pred):
    # Convert predicted probabilities to class labels
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    # Calculate the number of true positive, false positive, and false negative predictions for each class
    true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
    false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
    false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

    # Calculate precision and recall for each class
    precision = true_positives / (true_positives + false_positives + 1e-6)
    recall = true_positives / (true_positives + false_negatives + 1e-6)

    # Calculate the F1 score for each class
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    # Calculate the macro-averaged F1 score by taking the mean of the F1 scores for each class
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

##Model

In [8]:
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.metrics import Accuracy

In [9]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dim1, dim2, dim3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

with strategy.scope():
    inputs = tf.keras.Input(shape=(dim1, dim2, dim3))
    x = data_augmentation(inputs)
    x = layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dim1, dim2, dim3))(inputs)
    x = layers.experimental.preprocessing.RandomRotation(0.1)(x)
    x = layers.experimental.preprocessing.RandomZoom(0.1)(x)
    x = layers.Conv2D(16, 3, padding='valid', activation='elu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(16, 3, padding='valid', activation='elu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(16, 3, padding='valid', activation='elu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(16, 3, padding='valid', activation='elu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(16, 3, padding='valid', activation='elu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='elu')(x)
    x = layers.Dense(512, activation='elu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

normal_model = tf.keras.Model(inputs=inputs, outputs=outputs)

normal_model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=[macro_f1])
es = EarlyStopping(monitor='macro_f1', mode='max', verbose=1)


In [10]:
normal_model.fit(train_generator, epochs=20, callbacks=[es])

Epoch 1/20
209/209 [==============================] - 113s 445ms/step - loss: 1.7504 - macro_f1: 0.3556
Epoch 2/20
209/209 [==============================] - 95s 455ms/step - loss: 1.2568 - macro_f1: 0.3589
Epoch 3/20
209/209 [==============================] - 90s 429ms/step - loss: 1.1351 - macro_f1: 0.3600
Epoch 4/20
209/209 [==============================] - 91s 435ms/step - loss: 1.0253 - macro_f1: 0.3626
Epoch 5/20
209/209 [==============================] - 93s 445ms/step - loss: 0.9547 - macro_f1: 0.3668
Epoch 6/20
209/209 [==============================] - 90s 431ms/step - loss: 0.8535 - macro_f1: 0.3675
Epoch 7/20
209/209 [==============================] - 94s 448ms/step - loss: 0.7572 - macro_f1: 0.3724
Epoch 8/20
209/209 [==============================] - 92s 440ms/step - loss: 0.6558 - macro_f1: 0.3737
Epoch 9/20
209/209 [==============================] - 92s 441ms/step - loss: 0.5351 - macro_f1: 0.3763
Epoch 10/20
209/209 [==============================] - 90s 431ms/step - 

In [11]:
train_loss, train_fscore = normal_model.evaluate(train_generator)
print(train_fscore)
test_loss, test_fscore = normal_model.evaluate(test_generator)
print(test_fscore)

209/209 [==============================] - 83s 394ms/step - loss: 0.4930 - macro_f1: 0.3773
0.3773311972618103
94/94 [==============================] - 38s 402ms/step - loss: 2.4566 - macro_f1: 0.4083
0.40827950835227966


In [12]:
normal_model.save("model", include_optimizer=False, save_format='tf')

In [13]:
!zip -r 'model.zip' '/content/model'

  adding: content/model/ (stored 0%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/keras_metadata.pb (deflated 94%)
  adding: content/model/fingerprint.pb (stored 0%)
  adding: content/model/saved_model.pb (deflated 90%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/model/variables/variables.index (deflated 69%)


## Testing

In [ ]:
import dlib
import cv2

def hogDetectFaces(image, image_path):
    # Initialize HOG face detector only once
    hog_face_detector = dlib.get_frontal_face_detector()

    def detect_face(image):
        height, width, _ = image.shape
        output_image = image.copy()

        # OpenCV reads images in BGR format by default
        imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hog_face_detector(imgRGB, 0)

        # If 0 or more than 1 face is detected
        if len(results) != 1:
            return image_path

        # Some images have bounding box in their borders
        x1, y1 = max(0, results[0].left()), max(0, results[0].top())
        x2, y2 = max(0, results[0].right()), max(0, results[0].bottom())

        cropped_image_height, cropped_image_width = y2 - y1, x2 - x1

        if 71 < cropped_image_height < 643 and 68 < cropped_image_width < 642:
            return output_image[y1:y2, x1:x2]

        return image_path

    return detect_face(image)

image_path = '/content/1690657713992.jfif'
real_image = cv2.imread(image_path)
real_image = hogDetectFaces(real_image, image_path)

# Create a black image of size (642,641,3)
padded_image = np.zeros((642, 641, 3), dtype=np.uint8)
# Compute the center offset to place the cropped image on the black image
x_offset = (padded_image.shape[1] - real_image.shape[1]) // 2
y_offset = (padded_image.shape[0] - real_image.shape[0]) // 2
# Place the cropped image onto the black image
padded_image[y_offset:y_offset+real_image.shape[0], x_offset:x_offset+real_image.shape[1]] = real_image
real_image = padded_image / 255.0

# Assuming real_image is your image
real_image = real_image[np.newaxis, ...]

# Make the prediction
prediction = normal_model.predict(real_image)

1/1 [==============================] - 0s 151ms/step


In [ ]:
predicted_class = np.argmax(prediction[0])

if predicted_class == 0:
    output_range = '1-6'
elif predicted_class == 1:
    output_range = '7-12'
elif predicted_class == 2:
    output_range = '13-19'
elif predicted_class == 3:
    output_range = '20-26'
elif predicted_class == 4:
    output_range = '26-30'
else:
    output_range = 'Invalid class'

print(output_range)

7-12
